<a href="https://colab.research.google.com/github/atick-faisal/GRF-Synthesis-from-Motion-Trajectories/blob/main/src/training/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EEG Motion Artifact Removal using Deep Learning - Complete Pipeline

# Test GPU (Optional)
Before Starting, kindly check the available GPU from the Google Server, GPU model and other related information. It might help!

In [1]:
import torch
print("Is CUDA enabled GPU Available?", torch.cuda.is_available())
print("GPU Number:", torch.cuda.device_count())
print("Current GPU Index:", torch.cuda.current_device())
print("GPU Type:", torch.cuda.get_device_name(device=None))
print("GPU Capability:", torch.cuda.get_device_capability(device=None))
print("Is GPU Initialized yet?", torch.cuda.is_initialized())

Is CUDA enabled GPU Available? True
GPU Number: 1
Current GPU Index: 0
GPU Type: Tesla K80
GPU Capability: (3, 7)
Is GPU Initialized yet? True


# Connect to Google Drive

In [2]:
# from google.colab import drive
# drive.mount('/content/GDrive')

Move to the Target Directory

In [3]:
# %cd /content/GDrive/MyDrive/Colab_Notebooks/Research/EEG_IMU 

List the Files and Folders Located in the Current Directory

In [4]:
# !ls

## Evaluation of Predicting ABP Waveforms

Here, we present an interactive CLI to predict the ABP waveform from PPG signal from the test data. Ground truth, prediction from approximation network and refinement network are presented, and a comparison is also demonstrated


# Conling Repository

In [5]:
!git clone https://github.com/atick-faisal/GRF-Synthesis-from-Motion-Trajectories.git

Cloning into 'GRF-Synthesis-from-Motion-Trajectories'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 99 (delta 37), reused 58 (delta 14), pack-reused 0
Unpacking objects: 100% (99/99), done.


In [6]:
%cd /content/GRF-Synthesis-from-Motion-Trajectories/src/
!ls

/content/GRF-Synthesis-from-Motion-Trajectories/src
models	preprocessing  training  visualization


#Import Libraries

In [7]:
import os
import h5py
import scipy
import random
import pickle
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tqdm import tqdm
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_error
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from pathlib import Path

In [8]:
from models import UNet

In [9]:
%matplotlib inline
sns.set_style('white')

# End-to-End Pipeline - EEG Motion Artifact Correction using Deep Learning

## Prepare Dataset - k-Fold Cross Validation

In [10]:
!gdown --id "1mAUGcHMEjAwRy0DTU_h_0Jf6TBfrTzsb"

Downloading...
From: https://drive.google.com/uc?id=1mAUGcHMEjAwRy0DTU_h_0Jf6TBfrTzsb
To: /content/GRF-Synthesis-from-Motion-Trajectories/src/data.joblib
100% 412M/412M [00:07<00:00, 53.8MB/s]


In [11]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

data = joblib.load('data.joblib')
features = data['X']
target = data['y']

features_z = features[['lfal_z', 'ltam_z', 'lfcc_z', 'lfm1_z', 'lfm2_z', 'lfm5_z']]
target_z = target[['fz']]

scaler = StandardScaler()
scaler.fit(features_z)
features_z = scaler.transform(features_z)

scaler = StandardScaler()
scaler.fit(target_z)
target_z = scaler.transform(target_z)

X = features_z.reshape(-1, 1024, 6)
y = target_z.reshape(-1, 1024, 1)

print(X.shape)
print(y.shape)

(1143, 1024, 6)
(1143, 1024, 1)


In [12]:
features_z.shape

(1170432, 6)

In [13]:
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, y, test_size=0.2, random_state=42)
Y_Train = np.squeeze(Y_Train, axis=2)
Y_Test = np.squeeze(Y_Test, axis=2)
Y_Train.shape

(914, 1024)

## Garbage Collector

In [14]:
# import gc #Garbage Collector
# fl_Train = None
# fl_Test = None
# X_Test = None
# X_Train = None
# X_Val = None
# Y_Test_BP = None
# Y_Train_BP = None
# Y_Test_ABP = None
# Y_Train_ABP = None
# SBP_Train = None
# DBP_Train = None
# gc.collect()

## Segmentation Pipeline

### Prepare Label

Configurations

In [15]:
signal_length = 1024 # Length of each Segment
model_depth = 5 # Number of Level in the CNN Model
model_width = 64 # Width of the Initial Layer, subsequent layers start from here
kernel_size = 3 # Size of the Kernels/Filter
num_channel = 6 # Number of Channels in the Model
D_S = 1 # Turn on Deep Supervision
A_E = 0 # Turn on AutoEncoder Mode for Feature Extraction
problem_type = 'Regression'
output_nums = 1 # Number of Class for Classification Problems, always '1' for Regression Problems
'''Only required if the AutoEncoder Mode is turned on'''
feature_number = 1024 # Number of Features to be Extracted
'''Only required for MultiResUNet'''
alpha = 1 # Model Width Expansion Parameter

Prepare Dataset for Deep Supervision (if Activated)

In [16]:
def prepareTrainDict(y, model_depth, signal_length, model_name):
  def approximate(inp, w_len, signal_length):
    op = np.zeros((len(inp),signal_length//w_len))
    for i in range(0,signal_length,w_len):
      try:
        op[:,i//w_len] = np.mean(inp[:,i:i+w_len],axis=1)
      except Exception as e:
        print(e)
        print(i)
  	
    return op

  out = {}
  Y_Train_dict = {}
  out['out'] = np.array(y)
  Y_Train_dict['out'] = out['out']
  for i in range(1, (model_depth+1)):
    name = f'level{i}'
    if ((model_name == 'UNet') or (model_name == 'MultiResUNet') or (model_name == 'FPN')):
      out[name] = np.expand_dims(approximate(np.squeeze(y), 2**i, signal_length),axis = 2)
    elif ((model_name == 'UNetE') or (model_name == 'UNetP') or (model_name == 'UNetPP')):
      out[name] = np.expand_dims(approximate(np.squeeze(y), 2**0, signal_length),axis = 2)
    Y_Train_dict[f'level{i}'] = out[f'level{i}']
  
  return out, Y_Train_dict

In [17]:
model_name = 'UNet' # UNet or UNetPP
X_Train1 = X_Train
X_Test1 = X_Test
[Y_Train1, Y_Train_dict] = prepareTrainDict(Y_Train, model_depth, signal_length, model_name)
[Y_Test1, Y_Test_dict] = prepareTrainDict(Y_Test, model_depth, signal_length, model_name)

In [18]:
loss_weights = np.zeros(model_depth)

for i in range(0, model_depth):
   loss_weights[i] = 1-(i*0.1)
   
loss_weights

array([1. , 0.9, 0.8, 0.7, 0.6])

### Train Data

Build and Compile Model

In [19]:
if D_S == 0:
    # Build model for EEG Motion Artifact Removal - Deep Unet Architecture
    # AutoEncoder should be set at 'FALSE' for the Deep U-net model for Signal Reconstruction
    # D_S = 0, A_E = 0
    EEGIMU_Network = UNet(signal_length, model_depth, num_channel, model_width, kernel_size, problem_type=problem_type, output_nums=output_nums, ds=D_S, ae=A_E, alpha=alpha).UNet()
    EEGIMU_Network.compile(loss= 'mean_absolute_error', optimizer= 'adam', metrics= ['mean_squared_error','accuracy'])
    #
elif D_S == 1:
    # Build model for EEG Motion Artifact Removal - Deep Unet Architecture
    # AutoEncoder should be set at 'FALSE' for the Deep U-net model for Signal Reconstruction
    # D_S = 1, A_E = 0
    EEGIMU_Network = UNet(signal_length, model_depth, num_channel, model_width, kernel_size, problem_type=problem_type, output_nums=output_nums, ds=D_S, ae=A_E, alpha=alpha).UNet()
    EEGIMU_Network.compile(loss= 'mean_absolute_error', optimizer= 'adam', metrics= ['mean_squared_error'], loss_weights= loss_weights)

Import Trained Weights for a Headstart

In [20]:
# if D_S == 0:
#     EEGIMU_Network_Network.load_weights('trained_models/'+model_name+'_'+str(length)+'_'+str(model_width)+'_'+str(num_channel)+'_'+str(D_S)+'.h5') # Load Previously Trained Weights for Transfer Learning
# elif D_S == 1:
#     EEGIMU_Network_Network.load_weights('trained_models/'+model_name+'_'+str(length)+'_'+str(model_width)+'_'+str(num_channel)+'_'+str(D_S)+'.h5') # Load Previously Trained Weights for Transfer Learning

Train Model

In [ ]:
if D_S == 0:
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, mode='min'), 
                 tf.keras.callbacks.ModelCheckpoint('trained_models/'+model_name+'_'+str(signal_length)+'_'+str(model_width)+'_'+str(model_depth)+'_'+str(num_channel)+'_'+str(D_S)+'.h5', verbose=1, monitor='val_loss', save_best_only=True, mode='min')]
    history = EEGIMU_Network.fit(X_Train, Y_Train, epochs=300, batch_size=64, verbose=1, validation_split=0.2, shuffle=True, callbacks=callbacks)
    #
elif D_S == 1:
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_out_loss', patience=30, mode='min'), 
                 tf.keras.callbacks.ModelCheckpoint('trained_models/'+model_name+'_'+str(signal_length)+'_'+str(model_width)+'_'+str(model_depth)+'_'+str(num_channel)+'_'+str(D_S)+'.h5', verbose=1, monitor='val_out_loss', save_best_only=True, mode='min')]
    history = EEGIMU_Network.fit(X_Train1, Y_Train_dict, epochs=300, batch_size=64, verbose=1, validation_split=0.2, shuffle=True, callbacks=callbacks)

Epoch 1/300
12/12 [==============================] - 59s 1s/step - loss: 6.9376 - out_loss: 0.6569 - level1_loss: 0.6894 - level2_loss: 0.7479 - level3_loss: 1.1289 - level4_loss: 1.6615 - level5_loss: 3.7873 - out_mean_squared_error: 0.8540 - level1_mean_squared_error: 0.9785 - level2_mean_squared_error: 1.1965 - level3_mean_squared_error: 8.8441 - level4_mean_squared_error: 35.0506 - level5_mean_squared_error: 161.7185 - val_loss: 383.9533 - val_out_loss: 6.8015 - val_level1_loss: 73.3497 - val_level2_loss: 67.0839 - val_level3_loss: 304.2911 - val_level4_loss: 19.1489 - val_level5_loss: 9.2665 - val_out_mean_squared_error: 640.3518 - val_level1_mean_squared_error: 100468.2734 - val_level2_mean_squared_error: 71659.9609 - val_level3_mean_squared_error: 1009152.9375 - val_level4_mean_squared_error: 5011.4775 - val_level5_mean_squared_error: 569.5228

Epoch 00001: val_out_loss improved from inf to 6.80152, saving model to trained_models/UNet_1024_64_5_6_1.h5
Epoch 2/300
12/12 [========

Predict from the Test set

In [ ]:
if D_S == 0:
    EEG_App = EEGIMU_Network.predict(X_Test, verbose=1)
    print(EEG_App.shape)
elif D_S == 1:
    EEG_App = EEGIMU_Network.predict(X_Test1, verbose=1)
    print(EEG_App[0].shape)

Plot History Plots (e.g., Loss, Accuracy, etc.)

In [ ]:
def history_plot(history):
  # list all dictionaries in history
  print(history.history.keys())
  # summarize history for error
  plt.figure(figsize=(7,7))
  plt.subplot(2,1,1)
  plt.plot(history.history['out_mean_squared_error'])
  plt.plot(history.history['val_out_mean_squared_error'])
  plt.title('Model Error Performance')
  plt.ylabel('Error')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper right')
  plt.ylim([0, 3])
  plt.show()
  # summarize history for loss
  plt.figure(figsize=(7,7))
  plt.subplot(2,1,2)
  plt.plot(history.history['out_loss'])
  plt.plot(history.history['val_out_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper right')
  plt.ylim([0, 3])
  plt.show()
#
history_plot(history)

Visualize Outcome

In [ ]:
if D_S == 0:
    i = random.randint(0,len(Y_Test));
    #
    MAE = np.mean(np.abs(EEG_App[i].ravel()-Y_Test[i].ravel()));
    #
    plt.figure(figsize=(15, 4));
    plt.plot(Y_Test1[i],label='GRND');
    plt.plot(EEG_App[i].ravel(), label='Pred');
    plt.title(f"EEG -- Sample Number {i} -- MAE = {MAE}");
    plt.legend();
    #
elif D_S == 1:
    i = random.randint(0,len(Y_Test));
    #
    MAE = np.mean(np.abs(EEG_App[0][i].ravel()-Y_Test[i].ravel()))
    #
    plt.figure(figsize=(15,4))
    # plt.subplot(5,1,1)
    plt.plot(Y_Test[i],label='Ground Truth');
    plt.plot(EEG_App[0][i].ravel(), label='Prediction');
    plt.title(f"GRF -- Sample Number {i} -- MAE = {MAE}");
    plt.legend();
    plt.show()
    # plt.subplot(5,1,2)
    # plt.plot((X_Test[i,:,0].ravel()),label='EEG2');
    # plt.title('EEG2')
    # plt.legend();
    # plt.subplot(5,1,3)
    # plt.plot((X_Test[i,:,1].ravel()),label='ACCX2');
    # plt.title('ACCX')
    # plt.legend();
    # plt.subplot(5,1,4)
    # plt.plot((X_Test[i,:,2].ravel()),label='ACCY2');
    # plt.title('ACCY')
    # plt.legend();
    # plt.subplot(5,1,5)
    # plt.plot((X_Test[i,:,3].ravel()),label='ACCZ2');
    # plt.title('ACCZ')
    # plt.legend();

### Evaluate EEG Reconstruction

Construction Error

In [ ]:
def Construction_Error(GRND, Pred):
    construction_err = []
    bad_indices = []
    count = 0

    for i in range(len(GRND)):
        MAE = np.mean(np.abs(Pred[i].ravel() - GRND[i].ravel()))
        if MAE < 1:
            construction_err.append(MAE)
        elif MAE >= 1:
            count = count + 1
            bad_indices.append(i)

    print(f'Construction Error : {round(np.mean(construction_err), 3)} +/- {round(np.std(construction_err), 3)}')
    print(f'Number of Bad Predictions = {count}')

    bad_indices = set(bad_indices)
    all_indices = set(np.arange(len(GRND)))
    good_indices = np.array(list(all_indices - bad_indices))
    GRND_NEW = GRND[good_indices]
    PRED_NEW = Pred[good_indices]

    return GRND_NEW, PRED_NEW

In [ ]:
[A,B] = Construction_Error(Y_Test,EEG_App[0])

### Save Files

In [ ]:
output_directory = 'Patient_Wise_Data/Outputs/Preds_Sub_' + str(Test_Case) + '.h5'
File = h5py.File(output_directory, 'w')
File.create_dataset('EEG_GT', data=Y_Test)
File.create_dataset('EEG_MA', data=X_Test[:,:,0])
File.create_dataset('EEG_Pred', data=EEG_App[0])
File.close()

# Infinite Loop to Keep the Tab Alive

In [ ]:
while True:
    pass